In [1]:
import pandas as pd
import os
import torch
import time
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.metrics import *

%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
batch_size = 400
epochs = 120
max_lr = 0.001
grad_clip = 0.01
weight_decay =0.001
opt_func = torch.optim.Adam

In [3]:
train_data = torchvision.datasets.CIFAR100('./', train=True, download=True)

x = np.concatenate([np.asarray(train_data[i][0]) for i in range(len(train_data))]) #1600000 X 32 X 3 array
 
mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0, 1))/255
 
mean=mean.tolist()
std=std.tolist()


100%|██████████| 169001437/169001437 [00:06<00:00, 26205688.35it/s]


Extracting ./cifar-100-python.tar.gz to ./


In [4]:
train_transform = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         tt.Normalize(mean,std,inplace=True)])
test_transform = tt.Compose([tt.ToTensor(), tt.Normalize(mean,std)])

In [5]:
trainset = torchvision.datasets.CIFAR100("./",
                                         train=True,
                                         download=True,
                                         transform=train_transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size, shuffle=True, num_workers=2,pin_memory=True)

testset = torchvision.datasets.CIFAR100("./",
                                        train=False,
                                        download=True,
                                        transform=test_transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size*2,pin_memory=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Device Setup

In [6]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [7]:
device = get_default_device()
device

device(type='cuda')

In [8]:
trainloader = DeviceDataLoader(trainloader, device)
testloader = DeviceDataLoader(testloader, device)

# Layer

In [18]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
        
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True) 
        
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128)) 
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True) 
        
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512)) 
        self.conv5 = conv_block(512, 1028, pool=True) 
        self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))  
        
        self.classifier = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                        nn.Flatten(), # 1028 
                                        nn.Linear(1028, num_classes)) # 1028 -> 100
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.conv5(out)
        out = self.res3(out) + out
        out = self.classifier(out)
        return out

model = to_device(ResNet9(3, 100), device)
model

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

# Training The Model

In [19]:
@torch.no_grad()
def evaluate(model, test_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in test_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, test_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
   
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
 
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
           
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
           
            lrs.append(get_lr(optimizer))
            sched.step()
        
        result = evaluate(model, test_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [20]:
h = [evaluate(model, testloader)]
h

[{'val_loss': 4.605810642242432, 'val_acc': 0.009903847239911556}]

In [21]:
h += fit_one_cycle(int(epochs/4), max_lr, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

Epoch [0], last_lr: 0.00007, train_loss: 3.6116, val_loss: 3.0522, val_acc: 0.2601
Epoch [1], last_lr: 0.00015, train_loss: 2.8721, val_loss: 2.7676, val_acc: 0.3094
Epoch [2], last_lr: 0.00028, train_loss: 2.5340, val_loss: 2.5766, val_acc: 0.3416
Epoch [3], last_lr: 0.00044, train_loss: 2.2905, val_loss: 2.4500, val_acc: 0.3811
Epoch [4], last_lr: 0.00060, train_loss: 2.1004, val_loss: 2.4354, val_acc: 0.3817
Epoch [5], last_lr: 0.00076, train_loss: 1.9111, val_loss: 2.2123, val_acc: 0.4408
Epoch [6], last_lr: 0.00089, train_loss: 1.7705, val_loss: 2.4124, val_acc: 0.4156
Epoch [7], last_lr: 0.00097, train_loss: 1.6341, val_loss: 2.0725, val_acc: 0.4588
Epoch [8], last_lr: 0.00100, train_loss: 1.5078, val_loss: 2.3241, val_acc: 0.4178
Epoch [9], last_lr: 0.00099, train_loss: 1.3848, val_loss: 1.9250, val_acc: 0.4932
Epoch [10], last_lr: 0.00098, train_loss: 1.2841, val_loss: 1.6345, val_acc: 0.5584
Epoch [11], last_lr: 0.00095, train_loss: 1.1955, val_loss: 1.8463, val_acc: 0.5095
Ep

In [22]:
# Fitting the second 1/4 epochs
h += fit_one_cycle(int(epochs/4), max_lr/10, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

Epoch [0], last_lr: 0.00001, train_loss: 0.0454, val_loss: 0.9952, val_acc: 0.7430
Epoch [1], last_lr: 0.00002, train_loss: 0.0440, val_loss: 1.0023, val_acc: 0.7461
Epoch [2], last_lr: 0.00003, train_loss: 0.0437, val_loss: 1.0168, val_acc: 0.7406
Epoch [3], last_lr: 0.00004, train_loss: 0.0472, val_loss: 1.0521, val_acc: 0.7352
Epoch [4], last_lr: 0.00006, train_loss: 0.0516, val_loss: 1.0967, val_acc: 0.7264
Epoch [5], last_lr: 0.00008, train_loss: 0.0660, val_loss: 1.1268, val_acc: 0.7220
Epoch [6], last_lr: 0.00009, train_loss: 0.0851, val_loss: 1.1660, val_acc: 0.7052
Epoch [7], last_lr: 0.00010, train_loss: 0.0957, val_loss: 1.1777, val_acc: 0.7057
Epoch [8], last_lr: 0.00010, train_loss: 0.0980, val_loss: 1.2845, val_acc: 0.6933
Epoch [9], last_lr: 0.00010, train_loss: 0.0941, val_loss: 1.2104, val_acc: 0.7070
Epoch [10], last_lr: 0.00010, train_loss: 0.0831, val_loss: 1.1788, val_acc: 0.7093
Epoch [11], last_lr: 0.00010, train_loss: 0.0704, val_loss: 1.2001, val_acc: 0.7109
Ep

In [23]:

h += fit_one_cycle(int(epochs/8), max_lr/100, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

Epoch [0], last_lr: 0.00000, train_loss: 0.0082, val_loss: 1.0608, val_acc: 0.7451
Epoch [1], last_lr: 0.00000, train_loss: 0.0080, val_loss: 1.0618, val_acc: 0.7443
Epoch [2], last_lr: 0.00001, train_loss: 0.0084, val_loss: 1.0647, val_acc: 0.7421
Epoch [3], last_lr: 0.00001, train_loss: 0.0090, val_loss: 1.0627, val_acc: 0.7428
Epoch [4], last_lr: 0.00001, train_loss: 0.0095, val_loss: 1.0679, val_acc: 0.7419
Epoch [5], last_lr: 0.00001, train_loss: 0.0093, val_loss: 1.0611, val_acc: 0.7422
Epoch [6], last_lr: 0.00001, train_loss: 0.0097, val_loss: 1.0530, val_acc: 0.7458
Epoch [7], last_lr: 0.00001, train_loss: 0.0091, val_loss: 1.0553, val_acc: 0.7464
Epoch [8], last_lr: 0.00001, train_loss: 0.0081, val_loss: 1.0506, val_acc: 0.7448
Epoch [9], last_lr: 0.00000, train_loss: 0.0081, val_loss: 1.0514, val_acc: 0.7446
Epoch [10], last_lr: 0.00000, train_loss: 0.0077, val_loss: 1.0490, val_acc: 0.7437
Epoch [11], last_lr: 0.00000, train_loss: 0.0074, val_loss: 1.0463, val_acc: 0.7437
Ep

In [24]:

h += fit_one_cycle(int(epochs/8), max_lr/1000, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

Epoch [0], last_lr: 0.00000, train_loss: 0.0072, val_loss: 1.0471, val_acc: 0.7434
Epoch [1], last_lr: 0.00000, train_loss: 0.0074, val_loss: 1.0472, val_acc: 0.7435
Epoch [2], last_lr: 0.00000, train_loss: 0.0070, val_loss: 1.0478, val_acc: 0.7441
Epoch [3], last_lr: 0.00000, train_loss: 0.0071, val_loss: 1.0472, val_acc: 0.7444
Epoch [4], last_lr: 0.00000, train_loss: 0.0067, val_loss: 1.0473, val_acc: 0.7438
Epoch [5], last_lr: 0.00000, train_loss: 0.0070, val_loss: 1.0478, val_acc: 0.7433
Epoch [6], last_lr: 0.00000, train_loss: 0.0069, val_loss: 1.0484, val_acc: 0.7429
Epoch [7], last_lr: 0.00000, train_loss: 0.0067, val_loss: 1.0461, val_acc: 0.7437
Epoch [8], last_lr: 0.00000, train_loss: 0.0068, val_loss: 1.0472, val_acc: 0.7439
Epoch [9], last_lr: 0.00000, train_loss: 0.0067, val_loss: 1.0486, val_acc: 0.7435
Epoch [10], last_lr: 0.00000, train_loss: 0.0068, val_loss: 1.0490, val_acc: 0.7434
Epoch [11], last_lr: 0.00000, train_loss: 0.0066, val_loss: 1.0464, val_acc: 0.7438
Ep

In [25]:

h += fit_one_cycle(int(epochs/4), max_lr/100, model, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

Epoch [0], last_lr: 0.00000, train_loss: 0.0066, val_loss: 1.0455, val_acc: 0.7449
Epoch [1], last_lr: 0.00000, train_loss: 0.0067, val_loss: 1.0460, val_acc: 0.7426
Epoch [2], last_lr: 0.00000, train_loss: 0.0067, val_loss: 1.0441, val_acc: 0.7436
Epoch [3], last_lr: 0.00000, train_loss: 0.0069, val_loss: 1.0475, val_acc: 0.7437
Epoch [4], last_lr: 0.00001, train_loss: 0.0077, val_loss: 1.0443, val_acc: 0.7440
Epoch [5], last_lr: 0.00001, train_loss: 0.0080, val_loss: 1.0456, val_acc: 0.7431
Epoch [6], last_lr: 0.00001, train_loss: 0.0079, val_loss: 1.0507, val_acc: 0.7424
Epoch [7], last_lr: 0.00001, train_loss: 0.0081, val_loss: 1.0484, val_acc: 0.7426
Epoch [8], last_lr: 0.00001, train_loss: 0.0087, val_loss: 1.0503, val_acc: 0.7419
Epoch [9], last_lr: 0.00001, train_loss: 0.0089, val_loss: 1.0508, val_acc: 0.7409
Epoch [10], last_lr: 0.00001, train_loss: 0.0088, val_loss: 1.0502, val_acc: 0.7456
Epoch [11], last_lr: 0.00001, train_loss: 0.0087, val_loss: 1.0479, val_acc: 0.7419
Ep

# Prediction

In [26]:
# Saving the model to h5 file
torch.save(model.state_dict(), 'img_classification_cifar.h5')

In [27]:
# Generate testing accuracy, predicted label, confusion matrix, and table for classification report
def test_label_predictions(model, device, test_loader):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            prediction = output.argmax(dim=1, keepdim=True)
            actuals.extend(target.view_as(prediction))
            predictions.extend(prediction)
    return [i.item() for i in actuals], [i.item() for i in predictions]

y_test, y_pred = test_label_predictions(model, device, testloader)
cm=confusion_matrix(y_test, y_pred)
cr=classification_report(y_test, y_pred)
fs=f1_score(y_test,y_pred,average='weighted')
rs=recall_score(y_test, y_pred,average='weighted')
accuracy=accuracy_score(y_test, y_pred)
print('Confusion matrix:')
print(cm)
print(cr)
print('F1 score: %f' % fs)
print('Recall score: %f' % rs)
print('Accuracy score: %f' % accuracy)

Confusion matrix:
[[88  0  0 ...  0  0  0]
 [ 0 90  0 ...  0  0  0]
 [ 1  1 60 ...  0  4  0]
 ...
 [ 0  0  0 ... 78  0  0]
 [ 1  0  2 ...  0 63  0]
 [ 0  0  0 ...  0  0 75]]
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       100
           1       0.79      0.90      0.84       100
           2       0.67      0.60      0.63       100
           3       0.67      0.52      0.58       100
           4       0.57      0.67      0.61       100
           5       0.81      0.76      0.78       100
           6       0.75      0.79      0.77       100
           7       0.80      0.80      0.80       100
           8       0.91      0.86      0.89       100
           9       0.84      0.80      0.82       100
          10       0.62      0.55      0.58       100
          11       0.59      0.58      0.59       100
          12       0.80      0.83      0.81       100
          13       0.76      0.65      0.70       100
          14   